In [ ]:
# preprocess_masks.ipynb

import multiprocessing
from pathlib import Path
import nibabel as nib
import numpy as np
from scipy.ndimage import zoom
from tqdm import tqdm

# --- Configuration ---
RAW_DATA_ROOT = Path("/mnt/hot/public/COPDGene-1")
PROCESSED_DATA_DIR = Path("/mnt/hot/public/Akul/exhale_pred_data")
TARGET_SHAPE = (128, 128, 128)

def process_mask(patient_id: str):
    """
    Loads, resizes (using nearest-neighbor interpolation), and saves the exhale
    segmentation mask for a single patient ID.
    """
    try:
        # We only need the exhale mask for evaluating the exhale reconstruction
        mask_path = RAW_DATA_ROOT / f"{patient_id}_EXP_mask.nii.gz"

        if not mask_path.exists():
            return f"Skipped {patient_id}: Missing exhale mask file."

        mask_nii = nib.load(mask_path)
        mask_data = mask_nii.get_fdata().astype(np.uint8) # Masks are integer types
        
        # --- IMPORTANT ---
        # Resize the mask using nearest-neighbor interpolation (order=0) to
        # preserve the binary nature of the mask without creating intermediate values.
        zoom_factors = [t / s for t, s in zip(TARGET_SHAPE, mask_data.shape)]
        mask_resized = zoom(mask_data, zoom_factors, order=0, prefilter=False)
        
        # Save the processed mask
        save_dir = PROCESSED_DATA_DIR / "masks" / "exhale"
        np.save(save_dir / f"{patient_id}_EXP_mask.npy", mask_resized)
        
        return None
    except Exception as e:
        return f"Error processing {patient_id} mask: {e}"

def main():
    """
    Finds all patient IDs with exhale masks and processes them in parallel.
    """
    # Create the output directory
    (PROCESSED_DATA_DIR / "masks" / "exhale").mkdir(parents=True, exist_ok=True)

    all_mask_files = list(RAW_DATA_ROOT.glob("*_EXP_mask.nii.gz"))
    patient_ids = sorted([f.name.split('_')[0] for f in all_mask_files])

    if not patient_ids:
        print(f"Error: No exhale masks found in {RAW_DATA_ROOT}. Please check the path and file naming.")
        return

    print(f"Found {len(patient_ids)} patient exhale masks. Starting preprocessing...")
    
    num_processes = multiprocessing.cpu_count()
    print(f"Starting parallel processing with {num_processes} workers.")
    
    with multiprocessing.Pool(processes=num_processes) as pool:
        results = list(tqdm(pool.imap_unordered(process_mask, patient_ids), total=len(patient_ids)))

    error_count = sum(1 for res in results if res is not None)
    for res in results:
        if res is not None:
            print(res)

    print("\\n--- Mask Preprocessing Complete ---")
    print(f"Successfully processed: {len(patient_ids) - error_count} masks.")
    print(f"Failed to process: {error_count} masks.")

if __name__ == "__main__":
    main()

Found 8701 patient exhale masks. Starting preprocessing...
Starting parallel processing with 40 workers.


100%|██████████| 8701/8701 [08:55<00:00, 16.26it/s]


\n--- Mask Preprocessing Complete ---
Successfully processed: 8701 masks.
Failed to process: 0 masks.


In [ ]:
# preprocess_masks.ipynb

import multiprocessing
from pathlib import Path
import nibabel as nib
import numpy as np
from scipy.ndimage import zoom
from tqdm import tqdm

# --- Configuration ---
RAW_DATA_ROOT = Path("/mnt/hot/public/COPDGene-1")
PROCESSED_DATA_DIR = Path("/mnt/hot/public/Akul/exhale_pred_data")
TARGET_SHAPE = (128, 128, 128)

def process_mask(patient_id: str):
    """
    Loads, resizes (using nearest-neighbor interpolation), and saves the exhale
    segmentation mask for a single patient ID.
    """
    try:
        # We only need the exhale mask for evaluating the exhale reconstruction
        mask_path = RAW_DATA_ROOT / f"{patient_id}_EXP_mask.nii.gz"

        if not mask_path.exists():
            return f"Skipped {patient_id}: Missing exhale mask file."

        mask_nii = nib.load(mask_path)
        mask_data = mask_nii.get_fdata().astype(np.uint8) # Masks are integer types
        
        # --- IMPORTANT ---
        # Resize the mask using nearest-neighbor interpolation (order=0) to
        # preserve the binary nature of the mask without creating intermediate values.
        zoom_factors = [t / s for t, s in zip(TARGET_SHAPE, mask_data.shape)]
        mask_resized = zoom(mask_data, zoom_factors, order=0, prefilter=False)
        
        # Save the processed mask
        save_dir = PROCESSED_DATA_DIR / "masks" / "exhale"
        np.save(save_dir / f"{patient_id}_EXP_mask.npy", mask_resized)
        
        return None
    except Exception as e:
        return f"Error processing {patient_id} mask: {e}"

def main():
    """
    Finds all patient IDs with exhale masks and processes them in parallel.
    """
    # Create the output directory
    (PROCESSED_DATA_DIR / "masks" / "exhale").mkdir(parents=True, exist_ok=True)

    all_mask_files = list(RAW_DATA_ROOT.glob("*_EXP_mask.nii.gz"))
    patient_ids = sorted([f.name.split('_')[0] for f in all_mask_files])

    if not patient_ids:
        print(f"Error: No exhale masks found in {RAW_DATA_ROOT}. Please check the path and file naming.")
        return

    print(f"Found {len(patient_ids)} patient exhale masks. Starting preprocessing...")
    
    num_processes = multiprocessing.cpu_count()
    print(f"Starting parallel processing with {num_processes} workers.")
    
    with multiprocessing.Pool(processes=num_processes) as pool:
        results = list(tqdm(pool.imap_unordered(process_mask, patient_ids), total=len(patient_ids)))

    error_count = sum(1 for res in results if res is not None)
    for res in results:
        if res is not None:
            print(res)

    print("\\n--- Mask Preprocessing Complete ---")
    print(f"Successfully processed: {len(patient_ids) - error_count} masks.")
    print(f"Failed to process: {error_count} masks.")

if __name__ == "__main__":
    main()

Found 8701 patient exhale masks. Starting preprocessing...
Starting parallel processing with 40 workers.


100%|██████████| 8701/8701 [09:21<00:00, 15.49it/s]


\n--- Mask Preprocessing Complete ---
Successfully processed: 8701 masks.
Failed to process: 0 masks.


In [1]:
import os
import hashlib
from pathlib import Path
from tqdm.notebook import tqdm
import numpy as np

# --- Configuration ---
PROCESSED_DATA_DIR = Path("/mnt/hot/public/Akul/exhale_pred_data")
INHALE_MASKS_DIR = PROCESSED_DATA_DIR / "masks" / "inhale"
EXHALE_MASKS_DIR = PROCESSED_DATA_DIR / "masks" / "exhale"

def get_file_hash(filepath):
    """Computes the SHA256 hash of a file."""
    sha256 = hashlib.sha256()
    try:
        # Use np.load to ensure consistent hash for numpy arrays
        # regardless of system-level file metadata.
        data = np.load(filepath)
        sha256.update(data.tobytes())
        return sha256.hexdigest()
    except Exception as e:
        print(f"Could not read or hash file {filepath}: {e}")
        return None

def verify_and_rename():
    """
    Verifies that inhale masks are not duplicates of exhale masks
    and renames them from '_EXP_mask.npy' to '_INSP_mask.npy'.
    """
    print(f"Scanning directory: {INHALE_MASKS_DIR}")
    
    files_to_rename = [f for f in os.listdir(INHALE_MASKS_DIR) if f.endswith('_EXP_mask.npy')]
    
    if not files_to_rename:
        print("\nNo files with '_EXP_mask.npy' found in the inhale directory. It seems they may have already been renamed.")
        return

    renamed_count = 0
    identical_count = 0
    error_count = 0

    print(f"\nFound {len(files_to_rename)} potentially misnamed files. Verifying and renaming...")

    for filename in tqdm(files_to_rename, desc="Processing files"):
        misnamed_inhale_path = INHALE_MASKS_DIR / filename
        corresponding_exhale_path = EXHALE_MASKS_DIR / filename

        if not corresponding_exhale_path.exists():
            print(f"Warning: No corresponding exhale mask found for {filename}. Skipping.")
            error_count += 1
            continue

        # Compare file contents using a hash
        inhale_hash = get_file_hash(misnamed_inhale_path)
        exhale_hash = get_file_hash(corresponding_exhale_path)

        if inhale_hash is None or exhale_hash is None:
            error_count += 1
            continue

        if inhale_hash != exhale_hash:
            # Files are different, proceed with renaming
            new_filename = filename.replace('_EXP_mask.npy', '_INSP_mask.npy')
            new_inhale_path = INHALE_MASKS_DIR / new_filename
            os.rename(misnamed_inhale_path, new_inhale_path)
            renamed_count += 1
        else:
            # Files are identical, this is a problem
            print(f"--> CRITICAL WARNING: Inhale mask '{filename}' is identical to the exhale mask. Not renaming.")
            identical_count += 1

    print("\n--- Verification and Renaming Complete ---")
    print(f"Successfully renamed: {renamed_count} files.")
    print(f"Identical files found (skipped): {identical_count}")
    print(f"Errors or missing counterparts: {error_count}")

# --- Run the function ---
verify_and_rename()


Scanning directory: /mnt/hot/public/Akul/exhale_pred_data/masks/inhale

Found 8701 potentially misnamed files. Verifying and renaming...


Processing files:   0%|          | 0/8701 [00:00<?, ?it/s]


--- Verification and Renaming Complete ---
Successfully renamed: 8701 files.
Identical files found (skipped): 0
Errors or missing counterparts: 0
